In [1]:
# Import pandas & yfinance
import pandas as pd
import yfinance as yf
# Import numpy
import numpy as np
from numpy.linalg import multi_dot
# Set numpy random seed
np.random.seed(23)
# Import cufflinks
import cufflinks as cf

cf.set_config_file(offline=True, dimensions=(1000, 600))
# Import plotly express for EF plot
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

In [2]:
# Specify assets / stocks
# international etf portfolio : 'SPY', 'GLD', 'IWM', 'VWO', 'BND']
# indian stocks : bank, consumer goods, diversified, it, consumer durables
# ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets = ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets.sort()
# Number of assets
numofasset = len(assets)
# Number of portfolio for optimization
numofportfolio = 5000

In [3]:
# Get yahoo tickers for indian stocks
# yahooticker = [x+'.NS' for x in assets]
# Fetch / read data for multiple stocks at once
# df = yf.download(yahooticker, start='2015-01-01', end='2022-12-31', →progress=False)['Adj Close']
# df.columns = assets
# write data to file for future use
# df.to_csv('data/india_stocks.csv')
# Read from file
df = pd.read_csv('./india_stocks.csv', index_col=0, parse_dates=True)
# Display dataframe
df

,ASIANPAINT,HDFCBANK,ITC,RELIANCE,TCS
Date,,,,,
2015-01-01 00:00:00+05:30,709.6138,451.4752,192.3574,417.9889,1075.8510
2015-01-02 00:00:00+05:30,734.1470,457.7586,192.9599,416.8826,1090.1781
2015-01-05 00:00:00+05:30,734.0996,453.8938,193.7196,412.3163,1073.6107
2015-01-06 00:00:00+05:30,716.5759,446.8280,188.7424,393.6035,1034.0306
2015-01-07 00:00:00+05:30,730.9738,448.1321,185.2321,402.1714,1021.8162
...,...,...,...,...,...
2022-12-26 00:00:00+05:30,3056.0500,1629.4500,329.4050,2524.0500,3188.3152
2022-12-27 00:00:00+05:30,3112.6001,1631.1000,327.9775,2544.7000,3194.7844
2022-12-28 00:00:00+05:30,3123.7000,1629.8000,329.2081,2544.4500,3192.4321


In [7]:
# Plot price history
df['2022':].normalize().iplot(kind='line')

In [6]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100,2)
annual_stdev = round(returns.std()*np.sqrt(260)*100,2)
df1 = pd.DataFrame({
'Ann Ret': annual_returns,
'Ann Vol': annual_stdev
})
df1

,Ann Ret,Ann Vol
ASIANPAINT,22.98,26.88
HDFCBANK,19.71,23.75
ITC,10.63,27.00
RELIANCE,28.23,29.81
TCS,17.39,24.77


In [8]:
# Plot annualized return and volatility
df1.iplot(
kind='bar',
shared_xaxes=True,
orientation='h'
)

In [9]:
df1.reset_index().iplot(
kind='pie',
labels='index',
values='Ann Ret',
textinfo='percent+label',
hole=0.4
)


In [10]:
def portfolio_simulation(returns):
    # Initialize the lists
    rets = []; vols = []; wts = []
    # Simulate 5,000 portfolios
    for i in range (numofportfolio):
        # Generate random weights
        weights = np.random.random(numofasset)
        # Set weights such that sum of weights equals 1
        weights /= np.sum(weights)
        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean() * 260))
        vols.append(np.sqrt(multi_dot([weights.T, returns.cov()*260, weights])))
        wts.append(weights)
        # Create a dataframe for analysis
        data = {'port_rets': rets, 'port_vols': vols}
    for counter, symbol in enumerate(returns.columns.tolist ()) :
        data[symbol+' weight'] = [w[counter] for w in wts]
        portdf = pd.DataFrame(data)
        portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']
    return round(portdf,4)

In [11]:
# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp.head()


,port_rets,port_vols,ASIANPAINT weight,HDFCBANK weight,ITC weight,RELIANCE weight,TCS weight,sharpe_ratio
0,0.1848,0.1760,0.1893,0.3465,0.2801,0.1033,0.0809,1.0498
1,0.2102,0.1805,0.3015,0.0734,0.1724,0.2716,0.1810,1.1650
2,0.1714,0.1775,0.0009,0.3322,0.3325,0.1129,0.2215,0.9660
3,0.2151,0.1855,0.3959,0.3420,0.0263,0.1193,0.1165,1.1599
4,0.1951,0.1769,0.3287,0.2503,0.0442,0.0002,0.3766,1.1028


In [12]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets            0.2231
port_vols            0.1835
ASIANPAINT weight    0.2465
HDFCBANK weight      0.2125
ITC weight           0.0131
RELIANCE weight      0.2893
TCS weight           0.2385
sharpe_ratio         1.2159
Name: 2108, dtype: float64

In [13]:
# Plot simulated portfolio
fig = px.scatter(
    temp,
    x='port_vols',
    y='port_rets',
    color='sharpe_ratio',
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
).update_traces(mode='markers', marker=dict(symbol='cross'))
# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)
# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()
